# (Q)分子構造+(R)理由+(A)物性データセットのLLMによる学習と予測
- Q&A: 融点データセットを使用
- R: 自分自身で考えさせて､正解のデータを学習させる

In [1]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from transformers import AutoTokenizer
import pandas as pd
import random
import copy
import glob
import json
from datetime import datetime
from llmchem.utils import mk_dir,clean_vram

#import clear_output

from IPython.display import clear_output

/home/setup/miniconda3/envs/chemllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#dataset settings
n_test=50 #number of testing data
n_train_check=50 #number of training data for checking (i.e., checking everything takes too long, so we check only a part of training data)
n_GPT_reasoning=10 # number of reasoning data made by GPT
n_generation_iterations=300   # trial numbers to generate new self reasoning data
max_generations=10**5

#model settings
model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
target_modules= [
    "lm_head",
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate",
    "w1",
    "w2",
    "w3"
]

model_name=f"meta-llama/Llama-2-7b-chat-hf"
target_modules= [
    #"embed_tokens",
    "lm_head",
    #"q_proj",
    #"k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    #"down_proj",
]



#LoRA settings
r=32
lora_alpha=r
bit=16
#bit=8
#bit=4

#train settings
gradient_checkpointing =False
per_device_train_batch_size=1
epochs=3
lr=10**-5

#device settings
device_map="auto"

#dataset path
dataset_path="dataset/231225AutoReasoning/240117best_reason_record_3k.csv"

#project path
project_dir=f"results/projects/240118llama7b_{n_GPT_reasoning}"

#reasoning options
error_threshold=30  # if abolute error is smaller than this, add to training data

In [3]:
mk_dir(project_dir)
mk_dir(project_dir+"/eval")
mk_dir(project_dir+"/self_reasoning")
mk_dir(project_dir+"/train")

In [4]:
from llmchem.model import init_model
from llmchem.train import train_model
from llmchem.eval import eval_model
from llmchem.reasoning import self_reasoning
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
#load base dataset

df=pd.read_csv(dataset_path)
dataset=df.to_dict(orient="records")
random.seed(0)
random.shuffle(dataset)

base_train_dataset=dataset[:-n_test]
example_reasoning_dataset=base_train_dataset[:n_GPT_reasoning]
test_dataset=dataset[-n_test:]

In [6]:
#Loop: training, evaluation, data generation
for generation in range(max_generations):
    clear_output()
    print(f"Generation: {generation}")
    #prepare train dataset

    ## reason data made by GPT4
    train_dataset=copy.deepcopy(example_reasoning_dataset)

    print(f"GPT-generated reasons: {len(train_dataset)}")

    ## reason data made by model itself
    for path in glob.glob(f"{project_dir}/self_reasoning/*.json"):
        with open(path) as f:
            train_dataset.append(json.load(f))

    print(f"All-generated reasons: {len(train_dataset)}")
    random.shuffle(train_dataset)

    #train model
    clean_vram()
    model=init_model(model_name, r, lora_alpha, target_modules, bit=bit,device_map=device_map)
    train_result=train_model(model,tokenizer,train_dataset,
                    project_dir=project_dir,
                    epochs=epochs,
                    lr=lr,
                    per_device_train_batch_size=per_device_train_batch_size,
                    gradient_checkpointing=gradient_checkpointing,
                    )

    #eval

    train_check_dataset=copy.deepcopy(train_dataset[:n_train_check])
    random.shuffle(train_check_dataset)
    train_eval_result=eval_model(model,tokenizer,train_check_dataset,
                                f"{project_dir}/eval",
                                n_prompt_examples=3,
                                prompt_dataset=example_reasoning_dataset,
                                prefix=f"train_{generation}"
                                )

    test_eval_result=eval_model(model,tokenizer,test_dataset,
                                f"{project_dir}/eval",
                                n_prompt_examples=3,
                                prompt_dataset=example_reasoning_dataset,
                                prefix=f"test_{generation}"
                                )

    #generate additional training data by self-reasoning
    self_reasoning(model,tokenizer,base_train_dataset,
                example_reasoning_dataset,project_dir,
                generation=generation,
                n_iterations=n_generation_iterations,
                error_threshold=error_threshold,
                n_max_trials=2)

Generation: 0
GPT-generated reasons: 10
All-generated reasons: 10
Using fp16 mode


The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


Map: 100%|██████████| 10/10 [00:00<00:00, 981.84 examples/s]
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


  0%|          | 0/10 [00:00<?, ?it/s]

...

  6%|▌         | 17/300 [12:38<5:07:20, 65.16s/it]

----


- Methyl group: The methyl group is a small, nonpolar group that adds steric bulk and increases molecular weight. It typically raises the melting point by approximately +20°C.
- Hydrazine: Hydrazine is a polar functional group that can form hydrogen bonds, which increases the melting point. Estimate an increase of approximately +60°C.
- Total estimated impact on melting point from all functional groups is: +20 (methyl group) + 60 (hydrazine) = 80°C.
##Prediction: 80.0


#Problem
actual:  -52.4 predicted:  80.0 err:  132.4
- Methyl group: The methyl group is a small, nonpolar group that adds steric bulk and increases molecular weight. It typically raises the melting point by approximately +20°C.
- Hydrazine: Hydrazine is a polar functional group that can form hydrogen bonds, which increases the melting point. Estimate an increase of approximately +60°C.
- Total estimated impact on melting point from all functional groups is: +20 (methyl group) + 60 (hydrazine) = 80°C.
----


##Pr

  6%|▌         | 18/300 [12:46<3:44:08, 47.69s/it]

----


##Prediction: 60.0


#Problem
----


##Prediction: 135.0


#Problem


  6%|▋         | 19/300 [13:06<3:04:45, 39.45s/it]